# Union Simulation

What do we actually want to do here? We want to understand how the network and liqudity is consumed by the network.
We need to loop a bunch of times, select a random agents, select a random action and select random inputs for said action. 

Things we want to measure
- total liqidity (pool balance - total borrowed)
- total members (with at least "effective_number" of vouches)
- total_borrowed
- total_repaid

In [1]:
%matplotlib inline  

import random
import matplotlib.pyplot as plt

random.seed(10)

agents = 100
effective_number = 1

borrow_probability = 10
vouch_probability = 35
repay_probability = 50
stake_probability = 5

max_stake = 1000
max_vouch = 1000
max_borrow = 1000 
min_borrow = 900
max_repay = 1000
min_repay = 900

In [2]:
action_names = ["borrow", "repay", "stake", "vouch"]

actions = {
    "borrow": {
        "probability": borrow_probability,
        "min": min_borrow,
        "max": max_borrow,
    },
    "repay": {
        "probability": repay_probability,
        "min": min_repay,
        "max": max_repay,
    },
    "stake": {
        "probability": stake_probability,
        "min": 0,
        "max": max_stake
    },
    "vouch": {
        "probability": vouch_probability,
        "min": 0,
        "max": max_vouch
    }
}

# Simulation

In [ ]:
simulation = []
connections = []

stakes = [0 for _ in range(agents)]
vouches = [{} for _ in range(agents)]
borrows = [0 for _ in range(agents)]
locks = [0 for _ in range(agents)]

snapshots_per_epoch = []

def get_total_borrowed():
    return sum(borrows)

def get_total_repaid():
    return sum(item["amount"] if item["action"] == "repay" else 0 for item in simulation)

def get_total_vouches(borrower):
    n_vouches = 0
    for i, vouch in enumerate(vouches):
        if borrower in vouch:
            n_vouches+=1
    return n_vouches

def get_total_members():
    return sum(int(get_total_vouches(agent) >= 3) for agent in range(agents))

def get_total_staked():
    return sum(item["amount"] if item["action"] == "stake" else 0 for item in simulation)

def get_total_liquidity():
    return get_total_staked() - get_total_borrowed()

def get_credit_limit(borrower):
    credit_limit = 0
    for staker, vouch in enumerate(vouches):
        if borrower in vouch:
            staked = stakes[staker]
            locked = locks[staker]
            n_vouch = vouch[borrower]
            credit_limit += min(staked - locked, n_vouch)
    return credit_limit

def get_total_outstanding(borrower):
    return borrows[borrower]

def update_locks(staker, amount, lock):
    credit_limit = 0
    remaining = amount
    for staker, vouch in enumerate(vouches):
        if borrower in vouch:
            staked = stakes[staker]
            locked = locks[staker]
            n_vouch = vouch[borrower]
            if lock:
                locks[staker] += min([staked, amount, n_vouch])
                remaining -= min([staked, amount, n_vouch])
            else:
                locks[staker] -= min([locked, amount])
                remaining -= min([locked, amount])

for i in range(10000):
    action_name = action_names[random.randint(0,3)]
    action = actions[action_name]
    
    do_action = random.randint(0,100) < action["probability"]
    
    staker = random.randint(0, agents - 1)
    borrower = random.randint(0, agents - 1)
    while borrower == staker:
        borrower = random.randint(0, agents - 1)
        
    amount = random.randint(action["min"], action["max"])
    
    if do_action: 
        if action_name == "vouch":
            connections.append({ "staker": staker, "borrower": borrower })
            if borrower not in vouches[staker]:
                vouches[staker][borrower] = amount
        elif action_name == "stake":
            stakes[staker] += amount 
        elif action_name == "borrow":
            credit_limit = get_credit_limit(borrower)
            amount = min([amount, credit_limit])
            borrows[borrower] += amount
            update_locks(staker, amount, True)
        elif action_name == "repay":
            outstanding = get_total_outstanding(borrower)
            amount = min([outstanding, amount])
            borrows[borrower] -= amount
            update_locks(staker, amount, False)
        
        simulation.append({
            "action": action_name,
            "amount": amount,
            "total_borrowed": get_total_borrowed(),
            "total_repaid": get_total_repaid(),
            "total_members": get_total_members(),
            "total_liquidity": get_total_liquidity(),
        })

## Graphs

In [ ]:
import matplotlib.pyplot as plt

y_total_liquidity = [item["total_liquidity"] for item in simulation]
x_epochs = [i for i, _ in enumerate(simulation)]

plt.plot(x_epochs, y_total_liquidity)
plt.xlabel("epochs")
plt.ylabel("total liquidity")
plt.title("Total Liquidity")
plt.show()

In [ ]:
y_total_liquidity = [item["total_borrowed"] for item in simulation]
x_epochs = [i for i, _ in enumerate(simulation)]

plt.plot(x_epochs, y_total_liquidity)
plt.xlabel("epochs")
plt.ylabel("total borrowed")
plt.title("Total Borrowed")
plt.show()

In [ ]:
y_total_liquidity = [item["total_repaid"] for item in simulation]
x_epochs = [i for i, _ in enumerate(simulation)]

plt.plot(x_epochs, y_total_liquidity)
plt.xlabel("epochs")
plt.ylabel("total repaid")
plt.title("Total Repaid")
plt.show()

In [ ]:
y_total_liquidity = [item["total_members"] for item in simulation]
x_epochs = [i for i, _ in enumerate(simulation)]

plt.plot(x_epochs, y_total_liquidity)
plt.xlabel("epochs")
plt.ylabel("total members")
plt.title("Total Members")
plt.show()